<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG19 Hyper Parameter Tuning

This notebook is used to Hyper Parameter Tune VGG19 model for glacoma diagnosis.

In [1]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules

# if IN_COLAB:
#     !pip install deriva
#     !pip install bdbag
#     !pip install --upgrade --force pydantic
#     !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml

In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [2]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-05-13 15:00:37,619 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-05-13 15:00:37,620 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [4]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-AGBG" # rid
# Change the confi_file with bag_url=["minid: train", "minid: Valid", "minid: test"]


In [5]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-05-13 15:00:37,672 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-05-13 15:00:37,672 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [6]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-05-13 15:00:39,629 - INFO - File [/data/munimadu/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_hyper_paramter_tuning_sreenidhi_may_12_2024.json] transfer successful. 0.84 KB transferred. Elapsed time: 0:00:00.000055.
2024-05-13 15:00:39,630 - INFO - Verifying MD5 checksum for downloaded file [/data/munimadu/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_hyper_paramter_tuning_sreenidhi_may_12_2024.json]
2024-05-13 15:00:39,652 - INFO - Configuration validation successful!


{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/munimadu/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'VGG19_Catalog_Model_Hyper_paramter_tuning',
    'rid': '2-AGBM'}],
  'Execution_Asset_Type': [{'name': 'VGG19_Catalog_Hyper_parameter_tuning',
    'rid': '2-AGBP'}]},
 'execution_rid': '2-AGCM',
 'workflow_rid': '2-AGBT',
 'bag_paths': [PosixPath('/data/2-277G_6aa1a6861eee5a79bce4bf071065355f95a066c2a1ff326089d43048a7e0f185/Dataset_2-277G'),
  PosixPath('/data/2-277J_81c873a311aa6a67cf2eef44bd9056cb19181b299a6e44327ea3553616f18725/Dataset_2-277J'),
  PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')],
 'assets_paths': [],
 'configuration_path': PosixPath('/data/munimadu/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_hyper_paramter_tuning_sreenidhi_may_12_2024.json')}

In [7]:
configuration_records

ConfigurationRecord(caching_dir=PosixPath('/data'), working_dir=PosixPath('/data/munimadu/EyeAI_working'), vocabs={'Workflow_Type': [Term(name='VGG19_Catalog_Model_Hyper_paramter_tuning', rid='2-AGBM')], 'Execution_Asset_Type': [Term(name='VGG19_Catalog_Hyper_parameter_tuning', rid='2-AGBP')]}, execution_rid='2-AGCM', workflow_rid='2-AGBT', bag_paths=[PosixPath('/data/2-277G_6aa1a6861eee5a79bce4bf071065355f95a066c2a1ff326089d43048a7e0f185/Dataset_2-277G'), PosixPath('/data/2-277J_81c873a311aa6a67cf2eef44bd9056cb19181b299a6e44327ea3553616f18725/Dataset_2-277J'), PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')], assets_paths=[], configuration_path=PosixPath('/data/munimadu/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_hyper_paramter_tuning_sreenidhi_may_12_2024.json'))

In [8]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
train_dir = configuration_records.bag_paths[0] # path to the raw train dataset
validation_dir = configuration_records.bag_paths[1]
test_dir = configuration_records.bag_paths[2]

# train_cropped_image_path = path to training cropped images
train_cropped_image_path, train_cropped_csv = EA.create_cropped_images(str(train_dir),
                                                                     output_dir = str(EA.working_dir) +'/train',
                                                                       crop_to_eye=True)
validation_cropped_image_path, validation_cropped_csv = EA.create_cropped_images(str(validation_dir),
                                                                                 output_dir = str(EA.working_dir) +'/valid',
                                                                                 crop_to_eye=True)
test_cropped_image_path, test_cropped_csv = EA.create_cropped_images(str(test_dir),
                                                                     output_dir = str(EA.working_dir) +'/test',
                                                                     crop_to_eye=True)


In [9]:
output_path = str(EA.working_dir) + "/Execution_Assets/" + configuration_records.vocabs['Execution_Asset_Type'][0].name
os.mkdir(output_path)
output_path

'/data/munimadu/EyeAI_working/Execution_Assets/VGG19_Catalog_Hyper_parameter_tuning'

In [10]:
# @title Execute Training algorithm
from eye_ai.models.vgg19_hyper_parameter_tuning import main
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  main(train_path=train_cropped_image_path,
       valid_path=validation_cropped_image_path, 
       graded_test_path=test_cropped_image_path, 
       output_path = output_path,  
       n_trials = 50
          )
                    
# takes 18.41 hours to run (66287 seconds)

2024-05-13 15:12:28.170420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 15:12:28.170462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 15:12:28.173390: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-13 15:12:28.198083: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 15:12:30.154321: W tensorflow/compiler/tf2

Found 9065 images belonging to 2 classes.
Found 3033 images belonging to 2 classes.
Found 1094 images belonging to 2 classes.


/home/munimadu/Repos/eye-ai-ml/eye_ai/models/vgg19_hyper_parameter_tuning.py:103: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  brightness_range = trial.suggest_uniform('brightness_range', -0.05, 0.05)  # Changed to suggest_uniform for a float range
2024-05-13 15:12:32.650839: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 15:12:32.769911: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/lin

{'2SKC_No_Glaucoma': 0, '2SKA_Suspected_Glaucoma': 1}
{'2SKC_No_Glaucoma': 0, '2SKA_Suspected_Glaucoma': 1}


L355
2024-05-13 15:12:32.781436: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 15:12:32.784887: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 15:12:32.935002: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#

...

Best value: 0.7697870630775412, Best params: {'rotation_range': -2, 'width_shift_range': 0.041246611943877, 'height_shift_range': 0.09178175185007667, 'horizontal_flip': True, 'vertical_flip': True, 'zoom_range': -0.03288698236980494, 'brightness_range': -0.006611207735048985, 'use_class_weights': True, 'pooling': 'flatten', 'dense_layers': 2, 'units_layer_0': 64, 'activation_func_0': 'elu', 'batch_norm_0': True, 'dropout_0': 0.12282600983943151, 'units_layer_1': 128, 'activation_func_1': 'tanh', 'batch_norm_1': True, 'dropout_1': 0.35522833054288, 'fine_tune_at': 9, 'fine_tuning_learning_rate_adam': 8.894489649572761e-06, 'batch_size': 256}
Found 9065 images belonging to 2 classes.
Found 3033 images belonging to 2 classes.


/home/munimadu/Repos/eye-ai-ml/eye_ai/models/vgg19_hyper_parameter_tuning.py:103: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  brightness_range = trial.suggest_uniform('brightness_range', -0.05, 0.05)  # Changed to suggest_uniform for a float range


Found 1094 images belonging to 2 classes.
{'2SKC_No_Glaucoma': 0, '2SKA_Suspected_Glaucoma': 1}
{'2SKC_No_Glaucoma': 0, '2SKA_Suspected_Glaucoma': 1}
Epoch 1/100
284/284 [==============================] - 93s 300ms/step - loss: 0.7161 - roc_auc_score: 0.6491 - f1_score_normal: 0.5461 - accuracy_score: 0.6082 - val_loss: 0.6128 - val_roc_auc_score: 0.7239 - val_f1_score_normal: 0.5946 - val_accuracy_score: 0.6578
Epoch 2/100
284/284 [==============================] - 84s 290ms/step - loss: 0.5786 - roc_auc_score: 0.7718 - f1_score_normal: 0.6514 - accuracy_score: 0.7138 - val_loss: 1.6119 - val_roc_auc_score: 0.7733 - val_f1_score_normal: 0.5768 - val_accuracy_score: 0.4590
Epoch 3/100
284/284 [==============================] - 84s 289ms/step - loss: 0.5409 - roc_auc_score: 0.8053 - f1_score_normal: 0.6820 - accuracy_score: 0.7390 - val_loss: 0.7777 - val_roc_auc_score: 0.8206 - val_f1_score_normal: 0.6550 - val_accuracy_score: 0.6195
Epoch 4/100
284/284 [==============================]

[I 2024-05-14 11:50:00,250] Trial 49 finished with value: 0.751219512195122 and parameters: {'rotation_range': 8, 'width_shift_range': 0.05875203043302328, 'height_shift_range': 0.09226342422860084, 'horizontal_flip': True, 'vertical_flip': True, 'zoom_range': -0.04265072035369672, 'brightness_range': 0.0034162250553938216, 'use_class_weights': True, 'pooling': 'flatten', 'dense_layers': 1, 'units_layer_0': 128, 'activation_func_0': 'tanh', 'batch_norm_0': True, 'dropout_0': 0.09670161769720986, 'fine_tune_at': 6, 'fine_tuning_learning_rate_adam': 1.512599085722345e-05, 'batch_size': 32}. Best is trial 34 with value: 0.7697870630775412.


Best value: 0.7697870630775412, Best params: {'rotation_range': -2, 'width_shift_range': 0.041246611943877, 'height_shift_range': 0.09178175185007667, 'horizontal_flip': True, 'vertical_flip': True, 'zoom_range': -0.03288698236980494, 'brightness_range': -0.006611207735048985, 'use_class_weights': True, 'pooling': 'flatten', 'dense_layers': 2, 'units_layer_0': 64, 'activation_func_0': 'elu', 'batch_norm_0': True, 'dropout_0': 0.12282600983943151, 'units_layer_1': 128, 'activation_func_1': 'tanh', 'batch_norm_1': True, 'dropout_1': 0.35522833054288, 'fine_tune_at': 9, 'fine_tuning_learning_rate_adam': 8.894489649572761e-06, 'batch_size': 256}
Best trial :
 Value:  0.7697870630775412
 Params: 
    rotation_range: -2
    width_shift_range: 0.041246611943877
    height_shift_range: 0.09178175185007667
    horizontal_flip: True
    vertical_flip: True
    zoom_range: -0.03288698236980494
    brightness_range: -0.006611207735048985
    use_class_weights: True
    pooling: flatten
    dense_l

In [11]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)

2024-05-14 14:00:45,818 - INFO - Initializing uploader: GenericUploader v1.7.0 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2024-05-14 14:00:45,823 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-05-14 14:00:45,824 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-05-14 14:00:45,863 - INFO - Checking for updated configuration...
2024-05-14 14:00:45,993 - INFO - Updated configuration found.
2024-05-14 14:00:45,998 - INFO - Scanning files in directory [/data/munimadu/EyeAI_working/Execution_Assets/VGG19_Catalog_Hyper_parameter_tuning]...
2024-05-14 14:00:46,002 - INFO - Including file: [/data/munimadu/EyeAI_working/Execution_Assets/VGG19_Catalog_Hyper_parameter_tuning/vgg19_hyperparameter_study.pkl].
2024-05-14 14:00:46,003 - INFO - Including file: [/da

# 